In [1]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

In [3]:
session = requests_retry_session()

features = []

def get_data(page):
    data = {
        "page": page,
        "page_size": 500,
    }

    response = session.get('https://openapparel.org/api/facilities/', params=data)
    
    return response.json()

do_stuff = True
page = 1

while do_stuff:
    response = get_data(page)
    
    features.extend(response["features"])
    
    if response["next"] != None:
        page +=1
    else:
        do_stuff = False

In [5]:
len(features)

13688

In [6]:
features[0]

{'id': 'AL2019083EVYRMQ',
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [19.560102, 40.724353]},
 'properties': {'name': 'Italmek ShPK',
  'address': 'L1 Maj Fier',
  'country_code': 'AL',
  'oar_id': 'AL2019083EVYRMQ',
  'country_name': 'Albania'}}

In [8]:
processed_data = []

for d in features:
    newcomer = {
        "brand": "UNKNOWN",
        "company_type": "UNKONWN",
        "country": d["properties"]["country_name"],
        "name": d["properties"]["name"],
        "lat": d["geometry"]["coordinates"][0],
        "lon": d["geometry"]["coordinates"][1],
    }

    processed_data.append(newcomer)

import json
with open("data/open_apparrel_locations.json", "w") as f:
    data = {
        "factories": processed_data,
    }
    f.write(json.dumps(data))